# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [1]:
import xlwings as xw
# xw.__version__

max_breads = 7
wb = xw.Book('себестоимостьА_в1.xlsx')
sht = wb.sheets['Рецептура']
rcp_breads = sht.range('рцп_пшеничный_хлеб').value
breads = rcp_breads[3:3+max_breads-1]
bId = -1
for bread in breads:
    bId += 1
    if bread[1]: #  Name is set
        cost = sum([bread[i] * rcp_breads[len(rcp_breads)-3][i] for i in range(5, len(bread)-1) if not bread[i] is None])
        print(bread[1], cost)

Хлеб Венгерский 21.48
Хлеб "Барбари" 16.525
Багет парижский 17.423000000000002
Чиабатта 18.085


2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [4]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('reviews_sample.csv', header=0)
recipes = pd.read_csv('recipes_sample.csv', header=0)
reviews['date'] = pd.to_datetime(reviews['date'])
combined = pd.merge(recipes, reviews, left_on='id', right_on='recipe_id')
# Оставляем только полезные столбцы, потому что иначе слишком много лишних
combined = combined[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
combined.head()
# print(combined.shape)

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
2,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
3,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
4,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [5]:
def get5Percent(df):
    return df.sample(frac=0.05, random_state=1)

In [6]:
saveName = 'recipes.xlsx'
try:
    wbR = xw.Book(saveName)
    defaultName = None
    shtRc = wbR.sheets['Рецепты']
    shtRw = wbR.sheets['Отзывы']
except:
    wbR = xw.Book()
    defaultName = wbR.sheets[0].name
    shtRc = wbR.sheets.add('Рецепты')
    shtRw = wbR.sheets.add('Отзывы')

shtRc.clear()
shtRw.clear()

shtRc.range('A1').options(pd.DataFrame, index=False, header=1).value = get5Percent(recipes)
shtRw.range('A1').options(pd.DataFrame, index=False, header=1).value = get5Percent(reviews)

wbR.save(saveName)

if defaultName:
    wbR.sheets[defaultName].delete()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 